# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, RegexpTokenizer

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
def load_data(filepath, table):
    engine = create_engine('sqlite:///'+filepath)
    df = pd.read_sql_table(table, con=engine)
    X = df['message']
    y = df.iloc[:, 4:]
    return X, y

X, y = load_data('InsertDatabaseName.db', 'DisasterClassifier')

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    # Tokenize text
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    # Initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Iterate through each token
    clean_tokens = []
    for tok in tokens:
        # Lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# Train classifier
pipeline.fit(X_train, y_train)
# Predict test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def display_results(y_test, y_pred):
    accuracy = (y_pred == y_test).mean().mean()
    print('OVERALL ACCURACY: {0:.2f}% \n'.format(accuracy*100))
    results = pd.DataFrame(y_pred, columns=y_test.columns)
    for col in y_test.columns:
        print('CATEGORY: {}\n'.format(col))
        print(classification_report(y_test[col], results[col]))
    
display_results(y_test, y_pred)

OVERALL ACCURACY: 94.27% 

CATEGORY: related

             precision    recall  f1-score   support

          0       0.61      0.37      0.46      1563
          1       0.82      0.92      0.87      4944
          2       0.58      0.47      0.52        47

avg / total       0.77      0.79      0.77      6554

CATEGORY: request

             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5443
          1       0.84      0.37      0.51      1111

avg / total       0.88      0.88      0.86      6554

CATEGORY: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

CATEGORY: aid_related

             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3884
          1       0.75      0.52      0.61      2670

avg / total       0.74      0.73    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f95205dc620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [8]:
# Specify parameters for grid search
parameters = {
        'vect__max_df': (0.5, 0.75, 1.0),
        'clf__estimator__n_estimators': [10, 50, 100, 200, 300]}

# Create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
# Train classifier
cv.fit(X_train, y_train)
# Predict test data
y_pred = cv.predict(X_test)

display_results(y_test, y_pred)

OVERALL ACCURACY: 94.71% 

CATEGORY: related

             precision    recall  f1-score   support

          0       0.72      0.27      0.39      1563
          1       0.80      0.97      0.88      4944
          2       0.58      0.32      0.41        47

avg / total       0.78      0.80      0.76      6554

CATEGORY: request

             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5443
          1       0.89      0.43      0.58      1111

avg / total       0.89      0.89      0.88      6554

CATEGORY: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

CATEGORY: aid_related

             precision    recall  f1-score   support

          0       0.78      0.88      0.83      3884
          1       0.78      0.64      0.70      2670

avg / total       0.78      0.78    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
# Identify the best parameters
cv.best_params_

{'clf__estimator__n_estimators': 300, 'vect__max_df': 0.5}

In [10]:
# Identify the best parameters
cv.best_params_

{'clf__estimator__criterion': 'entropy',
 'clf__estimator__max_depth': 10,
 'clf__estimator__max_features': 'log2',
 'clf__estimator__min_samples_split': 2,
 'tfidf__use_idf': True,
 'vect__max_df': 0.5}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(DecisionTreeClassifier())))])

In [13]:
# Train classifier
pipeline.fit(X_train, y_train)
# Predict test data
y_pred = pipeline.predict(X_test)

display_results(y_test, y_pred)

OVERALL ACCURACY: 93.95% 

CATEGORY: related

             precision    recall  f1-score   support

          0       0.59      0.29      0.39      1563
          1       0.80      0.93      0.86      4944
          2       0.68      0.32      0.43        47

avg / total       0.75      0.78      0.75      6554

CATEGORY: request

             precision    recall  f1-score   support

          0       0.90      0.95      0.93      5443
          1       0.69      0.50      0.58      1111

avg / total       0.87      0.88      0.87      6554

CATEGORY: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

CATEGORY: aid_related

             precision    recall  f1-score   support

          0       0.74      0.79      0.77      3884
          1       0.67      0.60      0.63      2670

avg / total       0.71      0.71    

In [14]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# Build pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('best', TruncatedSVD()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

In [15]:
# Train classifier
pipeline.fit(X_train, y_train)
# Predict test data
y_pred = pipeline.predict(X_test)

display_results(y_test, y_pred)

OVERALL ACCURACY: 91.94% 

CATEGORY: related

             precision    recall  f1-score   support

          0       0.32      0.23      0.26      1563
          1       0.77      0.84      0.81      4944
          2       0.46      0.26      0.33        47

avg / total       0.66      0.69      0.67      6554

CATEGORY: request

             precision    recall  f1-score   support

          0       0.86      0.93      0.89      5443
          1       0.40      0.23      0.29      1111

avg / total       0.78      0.81      0.79      6554

CATEGORY: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

CATEGORY: aid_related

             precision    recall  f1-score   support

          0       0.63      0.70      0.66      3884
          1       0.47      0.39      0.43      2670

avg / total       0.56      0.58    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [11]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import libraries
import pickle
import re
import sys
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, RegexpTokenizer

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


def load_data(database_filepath):
    ''' Load data, transform it into a dataframe, 
    and return the text and labels as X and Y.
    Params
    ======
        database_filepath (str): filepath to SQLite database
    '''
    engine = create_engine('sqlite:///'+database_filepath)
    df = pd.read_sql_table('DisasterResponse', con=engine)
    X = df['message']
    y = df.iloc[:, 4:]
    
    return X, y


def tokenize(text):
    ''' Tokenize and clean the text.
    Params
    ======
        text (str): original message data
    '''
    # Normalize text
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    # Tokenize text
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    # Initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Iterate through each token
    clean_tokens = []
    for tok in tokens:
        # Lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens    


def build_model():
    ''' Make a machine learning pipeline with term frequency-inverse document
    frequency, random forest classification, and grid search for tuning parameters.
    '''
    # Build pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))])
    # Specify parameters for grid search
    parameters = {
        'vect__max_df': (0.5, 0.75, 1.0),
        'clf__estimator__criterion': ['entropy', 'gini'],
        'clf__estimator__min_samples_split': [2, 3, 4],
        'clf__estimator__n_estimators': [100, 200, 300]}
    # Create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=3)
    
    return cv


def evaluate_model(model, X_test, y_test):
    ''' Provide an evaluation for the machine learning model performance.
    Params
    ======
        model (str): machine learning pipeline object
        X_test (str): test features
        y_test (str): test labels
    '''
    # Predict test data
    y_pred = model.predict(X_test)
    # Calculate the overall accuracy of the model
    accuracy = (y_pred == y_test).mean().mean()
    print('OVERALL ACCURACY: {0:.2f}% \n'.format(accuracy*100))
    # Make a dataframe of the results
    results = pd.DataFrame(y_pred, columns=y_test.columns)
    # Display f1 score, precision, and recall for each disaster output category
    for column in y_test.columns:
        print('CATEGORY: {}\n'.format(column))
        print(classification_report(y_test[column], results[column]))


def save_model(model, model_filepath):
    ''' Save the trained machine learning model as a Pickle file. 
    Params
    ======
    model (str): machine learning pipeline object
    model_filepath (str): destination path to the Pickle file
    '''
    pickle.dump(model, open(model_filepath, 'wb'))


def main():
    ''' Extract data from SQLite database, train a machine learning classifier model
    on the data, evaluate model performance, and save the trained model.
    '''
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, y = load_data(database_filepath)
        # Perform train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, y_test)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()